In [1]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import cv2  # Assuming you have OpenCV installed
import torchvision
from torchvision.datasets import ImageFolder
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import itertools
import math
from vit_pytorch.efficient import ViT

In [2]:
# Required Parameters
dataset = "C:\\Study\\OJT\\Dataset\\UCF-101"          # Dataset path
dataset2 = "C:\\Study\\OJT\Dataset_Extraction\\UCF101\\Full"               # Dataset2 path
train_path = "C:\\Study\\OJT\Dataset_Extraction\\UCF101\\UCF101Train"            # Training path
test_path = "C:\\Study\\OJT\Dataset_Extraction\\UCF101\\UCF101Test"             # Testing path
no_of_frames = 10000                     # Total number of frames to be extracted
categories = os.listdir(dataset)        # Name of each class/category

In [3]:
# Creating dataset directory
try:
    os.mkdir(dataset2)
    print("Folder {} created...".format(dataset2))
except:
    print("A folder {} already exists...".format(dataset2))

A folder C:\Study\OJT\Dataset_Extraction\UCF101\Full already exists...


In [5]:
# Creating training_set directory
try:
    os.mkdir(train_path)
    print("Folder {} created...".format(train_path))
except:
    print("A folder {} already exists...".format(train_path))

A folder C:\Study\OJT\Dataset_Extraction\UCF101\UCF101Train already exists...


In [6]:
# Creating testing_set directory
try:
    os.mkdir(test_path)
    print("Folder {} created...".format(test_path))
except:
    print("A folder {} already exists...".format(test_path))

A folder C:\Study\OJT\Dataset_Extraction\UCF101\UCF101Test already exists...


In [7]:
# Creating same directories for dataset2/ that are already present in the dataset directory
for category in categories:
    try:
        os.mkdir(os.path.join(dataset2, category))
        print("Folder {} created...".format(category))
    except:
        print("A folder already exists, named {}...".format(category, dataset))

Folder ApplyEyeMakeup created...
Folder ApplyLipstick created...
Folder Archery created...
Folder BabyCrawling created...
Folder BalanceBeam created...
Folder BandMarching created...
Folder BaseballPitch created...
Folder Basketball created...
Folder BasketballDunk created...
Folder BenchPress created...
Folder Biking created...
Folder Billiards created...
Folder BlowDryHair created...
Folder BlowingCandles created...
Folder BodyWeightSquats created...
Folder Bowling created...
Folder BoxingPunchingBag created...
Folder BoxingSpeedBag created...
Folder BreastStroke created...
Folder BrushingTeeth created...
Folder CleanAndJerk created...
Folder CliffDiving created...
Folder CricketBowling created...
Folder CricketShot created...
Folder CuttingInKitchen created...
Folder Diving created...
Folder Drumming created...
Folder Fencing created...
Folder FieldHockeyPenalty created...
Folder FloorGymnastics created...
Folder FrisbeeCatch created...
Folder FrontCrawl created...
Folder GolfSwing 

In [8]:
for category in categories:
    try:
        os.mkdir(os.path.join(train_path, category))
        print("Folder {} created...".format(category))
    except:
        print("A folder already exists, named {}...".format(category, train_path))


Folder ApplyEyeMakeup created...
Folder ApplyLipstick created...
Folder Archery created...
Folder BabyCrawling created...
Folder BalanceBeam created...
Folder BandMarching created...
Folder BaseballPitch created...
Folder Basketball created...
Folder BasketballDunk created...
Folder BenchPress created...
Folder Biking created...
Folder Billiards created...
Folder BlowDryHair created...
Folder BlowingCandles created...
Folder BodyWeightSquats created...
Folder Bowling created...
Folder BoxingPunchingBag created...
Folder BoxingSpeedBag created...
Folder BreastStroke created...
Folder BrushingTeeth created...
Folder CleanAndJerk created...
Folder CliffDiving created...
Folder CricketBowling created...
Folder CricketShot created...
Folder CuttingInKitchen created...
Folder Diving created...
Folder Drumming created...
Folder Fencing created...
Folder FieldHockeyPenalty created...
Folder FloorGymnastics created...
Folder FrisbeeCatch created...
Folder FrontCrawl created...
Folder GolfSwing 

In [9]:
# Creating same directories for testing_set/ that are already present in the dataset directory
for category in categories:
    try:
        os.mkdir(os.path.join(test_path, category))
        print("Folder {} created...".format(category))
    except:
        print("A folder already exists, named {}...".format(category, test_path))


Folder ApplyEyeMakeup created...
Folder ApplyLipstick created...
Folder Archery created...
Folder BabyCrawling created...
Folder BalanceBeam created...
Folder BandMarching created...
Folder BaseballPitch created...
Folder Basketball created...
Folder BasketballDunk created...
Folder BenchPress created...
Folder Biking created...
Folder Billiards created...
Folder BlowDryHair created...
Folder BlowingCandles created...
Folder BodyWeightSquats created...
Folder Bowling created...
Folder BoxingPunchingBag created...
Folder BoxingSpeedBag created...
Folder BreastStroke created...
Folder BrushingTeeth created...
Folder CleanAndJerk created...
Folder CliffDiving created...
Folder CricketBowling created...
Folder CricketShot created...
Folder CuttingInKitchen created...
Folder Diving created...
Folder Drumming created...
Folder Fencing created...
Folder FieldHockeyPenalty created...
Folder FloorGymnastics created...
Folder FrisbeeCatch created...
Folder FrontCrawl created...
Folder GolfSwing 

In [10]:
# Combining multiple videos into single video file
for category in tqdm(categories):
    videofiles = glob.glob(os.path.join(dataset, category, "**/*.avi"), recursive=True)
    if videofiles:
        cap = cv2.VideoCapture(videofiles[0])
    else:
        print("No video files found in {}/{}".format(dataset, category))
    video_index = 0
    cap = cv2.VideoCapture(videofiles[0])    
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    # fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter("{}/{}/{}.avi".format(dataset2, category, category), fourcc, 25, (320, 240))
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
            video_index += 1
            if video_index >= len(videofiles):
                break
            else:
                cap = cv2.VideoCapture(videofiles[ video_index ])
                ret, frame = cap.read()
                out.write(frame)
        else:
            out.write(frame)
    cap.release()
    out.release()
    

  0%|          | 0/101 [00:00<?, ?it/s]

In [11]:
# Saving total no. of frames of each classes/categories into an array
total_frames = []
for category in tqdm(categories):
    cap = cv2.VideoCapture(dataset2 + "/" + category + "/" + category + ".avi")
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_frames.append(length)
    cap.release()
    
print(total_frames)

  0%|          | 0/101 [00:00<?, ?it/s]

[24358, 19761, 23690, 21740, 12822, 32365, 14944, 18497, 9954, 18966, 30324, 42936, 30289, 11179, 14356, 19519, 31260, 31835, 18340, 33005, 21216, 12132, 13495, 15844, 18811, 25841, 46503, 14570, 14751, 18887, 11947, 23001, 22895, 21748, 17701, 26019, 14842, 18505, 26896, 13796, 31785, 31361, 13084, 39338, 11855, 30115, 10387, 63340, 28035, 26124, 17487, 28920, 15747, 15704, 23093, 30592, 16290, 13868, 37671, 44077, 47926, 41461, 39933, 23076, 56258, 30210, 18383, 25083, 40890, 13778, 43334, 8404, 21145, 55036, 19649, 45329, 29090, 35660, 13004, 16486, 29146, 22995, 20818, 43208, 13159, 20736, 22721, 25320, 22108, 23179, 18256, 26376, 12524, 24002, 26694, 15884, 12836, 27697, 14391, 24312, 23350]


In [12]:
# Extracting one frame per five frame from the Videos
for category in tqdm(categories):
    count = 0    
    a = glob.glob(dataset2 + '/' + category + '/' + category +'.avi')
    for i in range(len(a)):
        cap = cv2.VideoCapture(a[i])
        frameRate = cap.get(5)
        while(cap.isOpened()):
            frameId = cap.get(1)
            ret, frame = cap.read()
            if (ret != True):
                break
            if (frameId % math.floor(frameRate) == 0):
                cv2.imwrite(train_path + '/' + category + '/{}_{}.jpg'.format(category, count), frame)
                count += 1
        cap.release()

  0%|          | 0/101 [00:00<?, ?it/s]

In [13]:
# Moving 150 random images from training_set into testing_set
for category in tqdm(categories):
    sub_file = [file for file in glob.glob(train_path +'/'+ category +'/'+ "*.jpg")]
    test_files = random.sample(sub_file, 50)
    for test_file in test_files:
        img = cv2.imread(test_file)
        os.remove(test_file)
        test_filename = os.path.basename(test_file)
        cv2.imwrite(test_path +'/' + category + '/' + test_filename , img)
        

  0%|          | 0/101 [00:00<?, ?it/s]

In [4]:
# Training settings
batch_size = 64
epochs = 20
lr = 0.001
gamma = 0.7
seed = 42

In [5]:
# Define the transformation for loading and preprocessing images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),  # You can adjust the crop size as needed
    transforms.ToTensor(),
])

In [6]:
# Create a custom dataset for the entire training set
full_dataset = ImageFolder(train_path, transform=transform)

In [7]:
# Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [8]:
# Create custom datasets for training and test
train_dataset = ImageFolder(train_path, transform=transform)
test_dataset = ImageFolder(test_path, transform=transform)

In [9]:
# Create data loaders for training and test sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [10]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

In [11]:
model = ViT(
    dim=128,
    image_size=224,
    patch_size=32,
    num_classes=101,
    transformer=efficient_transformer,
    channels=3,
)

In [12]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [13]:
# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if batch_idx % 10 == 9:  # Print every 10 batches
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {running_loss / 10:.4f}, Training Accuracy: {100 * correct / total:.2f}%")
            running_loss = 0.0

    scheduler.step()

    # Validation accuracy calculation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}, Training Accuracy: {100 * correct / total:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

# Evaluation on the test set
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        _, test_predicted = torch.max(test_outputs.data, 1)
        test_total += test_labels.size(0)
        test_correct += (test_predicted == test_labels).sum().item()

print(f"Test Accuracy: {100 * test_correct / test_total:.2f}%")
torch.save(model.state_dict(), 'ViT_UCF101_pth')

Epoch 1, Batch 10/1474, Loss: 4.6827, Training Accuracy: 1.09%
Epoch 1, Batch 20/1474, Loss: 4.6292, Training Accuracy: 1.56%
Epoch 1, Batch 30/1474, Loss: 4.4403, Training Accuracy: 1.98%
Epoch 1, Batch 40/1474, Loss: 4.3247, Training Accuracy: 2.89%
Epoch 1, Batch 50/1474, Loss: 4.2538, Training Accuracy: 3.84%
Epoch 1, Batch 60/1474, Loss: 4.1482, Training Accuracy: 4.61%
Epoch 1, Batch 70/1474, Loss: 4.0476, Training Accuracy: 5.07%
Epoch 1, Batch 80/1474, Loss: 3.9504, Training Accuracy: 5.80%
Epoch 1, Batch 90/1474, Loss: 3.8775, Training Accuracy: 6.49%
Epoch 1, Batch 100/1474, Loss: 3.9551, Training Accuracy: 6.77%
Epoch 1, Batch 110/1474, Loss: 3.8227, Training Accuracy: 7.39%
Epoch 1, Batch 120/1474, Loss: 3.7127, Training Accuracy: 7.89%
Epoch 1, Batch 130/1474, Loss: 3.7077, Training Accuracy: 8.40%
Epoch 1, Batch 140/1474, Loss: 3.6892, Training Accuracy: 8.73%
Epoch 1, Batch 150/1474, Loss: 3.5938, Training Accuracy: 9.17%
Epoch 1, Batch 160/1474, Loss: 3.5485, Training A